## Fama e Bliss (1987)

Usando os dados das LTNs

In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

Para o caso brasileiro, a base de dados que será usada são os preços da Letra do Tesouro Nacional (LTN), cujo pagamento de R\$ 1000 é feito após $t$ períodos. As LTNs farão o papel dos títulos americanos.

Os dados das LTNs estão em periodicidade mensal (último dia do mês), e abrangem todo o período a partir de Junho de 2012 até Outubro de 2022. Essa base de dados também foi obtida via Reuters com acesso concedido pelo departamento de Economia da PUC-Rio.

In [3]:
# lendo base de dados dos preços das LTNs
ltn_precos = pd.read_csv('ltn_precos.csv', sep=',', index_col=0)

In [4]:
# função que transforma a coluna de datas em datas de fato
ltn_precos.index = pd.to_datetime(ltn_precos.index)

In [5]:
ltn_precos

,BR6MT=RR,BR1YT=RR,BR2YT=RR,BR3YT=RR,Unnamed: 5,Unnamed: 6
2022-10-31,941.2195,888.250,829.531,996.3775,NaN,NaN
2022-09-30,966.9310,908.698,826.359,991.6280,NaN,NaN
2022-08-31,956.1750,898.608,815.204,976.0250,NaN,NaN
2022-07-31,944.5160,887.308,800.305,952.6970,NaN,NaN
2022-06-30,965.8870,878.349,792.686,990.9840,NaN,NaN
...,...,...,...,...,...,...
NaT,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# dropando a coluna da LTN com pagamento após 6 meses e duas colunas sem valores
ltn_precos = ltn_precos.drop(['BR6MT=RR', 'Unnamed: 5', 'Unnamed: 6'], axis=1)

In [7]:
# dropando os NaNs
ltn_precos = ltn_precos.loc[ltn_precos.index.dropna()]

Para ficar de acordo com o paper de Fama e Bliss (1987), divide-se o valor das LTNs por 1000 de forma que o pagamento agora, é de R\$1.

In [9]:
# normalizando os preços das LTNs para que o pagamento seja de R$1
ltn_precos = ltn_precos/1000

In [10]:
ltn_precos

,BR1YT=RR,BR2YT=RR,BR3YT=RR
2022-10-31,0.888250,0.829531,0.996378
2022-09-30,0.908698,0.826359,0.991628
2022-08-31,0.898608,0.815204,0.976025
2022-07-31,0.887308,0.800305,0.952697
2022-06-30,0.878349,0.792686,0.990984
...,...,...,...
2012-10-31,0.938520,0.850100,1.083364
2012-09-30,0.930100,0.873745,1.058065
2012-08-31,0.924305,0.866180,1.048780
2012-07-31,0.937410,0.860395,1.042515


In [12]:
# renomenado as colunas de acordo com o paper
ltn_precos = ltn_precos.rename(columns={'BR1YT=RR':'p(1:t)', 'BR2YT=RR':'p(2:t)', 'BR3YT=RR':'p(3:t)'})

Agora precisamos criar algumas taxas de retorno. No artigo Fama e Bliss (1987), definem o rendimento de um título que paga \$1 em $t+x$ comprado em $t$ como 

$$
r(x:t) = \ln\left[\frac{p(x:t+x)}{p(x:t)}\right]
$$

Como $p(x:t+x) = 1$

$$
r(x:t) = -\ln[{p(x:t)}]
$$

In [13]:
ltn_precos['r(1:t)'] = -np.log(ltn_precos['p(1:t)'])
ltn_precos['r(2:t)'] = -np.log(ltn_precos['p(2:t)'])
ltn_precos['r(3:t)'] = -np.log(ltn_precos['p(3:t)'])

Uma outra taxa que será necessária é $r(x:t+1)$, interpretada como o retorno de uma LTN comprada em $t+1$ que paga R\$1 em $t+x+1$. A equação que nos entrega essa taxa é dada por

$$
r(x:t+1) = \frac{[1+r(x-1:t)]}{[1+r(1:t)]} - 1
$$

In [14]:
ltn_precos['r(1:t+1)'] = ((1+ltn_precos['r(2:t)'])/(1+ltn_precos['r(1:t)'])) - 1
ltn_precos['r(2:t+1)'] = ((1+ltn_precos['r(3:t)'])/(1+ltn_precos['r(1:t)'])) - 1

A última taxa necessária para a construção das variáveis é $r(1:t+x)$ que representa o retorno da LTN comprada em $t+x$ que paga R\$1 em $t+x+1$ obtida através de

$$
r(1:t+x) = \frac{[1+r(x+1:t)]}{[1+r(x:t)]} -1
$$

In [15]:
ltn_precos['r(1:t+2)'] = ((1+ltn_precos['r(3:t)'])/(1+ltn_precos['r(2:t)'])) - 1

In [16]:
ltn_precos

,p(1:t),p(2:t),p(3:t),r(1:t),r(2:t),r(3:t),r(1:t+1),r(2:t+1),r(1:t+2)
2022-10-31,0.888250,0.829531,0.996378,0.118502,0.186895,0.003629,0.061147,-0.102703,-0.154408
2022-09-30,0.908698,0.826359,0.991628,0.095742,0.190726,0.008407,0.086684,-0.079704,-0.153116
2022-08-31,0.898608,0.815204,0.976025,0.106908,0.204317,0.024267,0.088001,-0.074660,-0.149504
2022-07-31,0.887308,0.800305,0.952697,0.119563,0.222762,0.048458,0.092178,-0.063511,-0.142549
2022-06-30,0.878349,0.792686,0.990984,0.129711,0.232328,0.009057,0.090835,-0.106801,-0.181178
...,...,...,...,...,...,...,...,...,...
2012-10-31,0.938520,0.850100,1.083364,0.063451,0.162401,-0.080071,0.093046,-0.134959,-0.208596
2012-09-30,0.930100,0.873745,1.058065,0.072463,0.134967,-0.056442,0.058280,-0.120195,-0.168647
2012-08-31,0.924305,0.866180,1.048780,0.078713,0.143663,-0.047628,0.060210,-0.117122,-0.167261
2012-07-31,0.937410,0.860395,1.042515,0.064635,0.150364,-0.041636,0.080525,-0.099819,-0.166904


A taxa forward é definida como

$$
f(x,x-1:t) = r(x:t) - r(x-1:t)
$$

Enquanto a taxa de retorno no caso brasileiro, da LTN comprada em $t$ e vendida em $t+1$ que pagaria R\$1 em $t+x-1$ é dada por

$$
h(x,x-1:t+1) = r(x:t) - r(x-1:t+1)
$$

In [17]:
ltn_precos['f(2,1:t)'] = ltn_precos['r(2:t)'] - ltn_precos['r(1:t)']
ltn_precos['f(3,2:t)'] = ltn_precos['r(3:t)'] - ltn_precos['r(2:t)']

ltn_precos['h(2,1:t+1)'] = ltn_precos['r(2:t)'] - ltn_precos['r(1:t+1)']
ltn_precos['h(3,2:t+1)'] = ltn_precos['r(3:t)'] - ltn_precos['r(2:t+1)']

A regressão da Tabela 1 é dada por

$$
h(x,x-1:t+1) - r(1:t) = \alpha + \beta \left[f(x,x-1:t) - r(1:t)\right] + u_2(t+1)
$$

onde o regressor é $f(x,x-1:t) - r(1:t)$ e o regressando $h(x,x-1:t+1) - r(1:t)$

In [18]:
ltn_precos['y'] = ltn_precos['h(2,1:t+1)'] - ltn_precos['r(1:t)']
ltn_precos['x'] = ltn_precos['f(2,1:t)'] - ltn_precos['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_precos).fit()

Para $x=2$, a regressão é


$$
h(2,1:t+1) - r(1:t) = \alpha + \beta \left[f(2,1:t) - r(1:t)\right] + u_2(t+1)
$$

In [19]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     27.82
Date:                Thu, 20 Oct 2022   Prob (F-statistic):           5.79e-07
Time:                        18:34:43   Log-Likelihood:                 493.91
No. Observations:                 125   AIC:                            -983.8
Df Residuals:                     123   BIC:                            -978.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0065      0.000     13.796      0.0

In [20]:
ltn_precos['y'] = ltn_precos['h(3,2:t+1)'] - ltn_precos['r(1:t)']
ltn_precos['x'] = ltn_precos['f(3,2:t)'] - ltn_precos['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_precos).fit()

Para $x=3$, a regressão é


$$
h(3,2:t+1) - r(1:t) = \alpha + \beta \left[f(3,2:t) - r(1:t)\right] + u_2(t+1)
$$

In [21]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.564
Method:                 Least Squares   F-statistic:                     161.3
Date:                Thu, 20 Oct 2022   Prob (F-statistic):           4.00e-24
Time:                        18:36:05   Log-Likelihood:                 434.29
No. Observations:                 125   AIC:                            -864.6
Df Residuals:                     123   BIC:                            -858.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0103      0.001     12.953      0.0

Para replicar o primeiro modelo da Tabela 3, basta alterar a variável $h(x,x-1:t+1)$ por $r(1:t+x-1)$, isso é

$$
r(1:t+x-1) - r(1:t) = \alpha + \beta \left[f(x,x-1:t) - r(1:t)\right] + u_2(t+1)
$$

In [22]:
ltn_precos['y'] = ltn_precos['r(1:t+1)'] - ltn_precos['r(1:t)']
ltn_precos['x'] = ltn_precos['f(2,1:t)'] - ltn_precos['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_precos).fit()

Para $x=2$, a regressão é


$$
r(1:t+1) - r(1:t) = \alpha + \beta \left[f(2,1:t) - r(1:t)\right] + u_2(t+1)
$$

In [23]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     6055.
Date:                Thu, 20 Oct 2022   Prob (F-statistic):          1.80e-106
Time:                        18:36:52   Log-Likelihood:                 493.91
No. Observations:                 125   AIC:                            -983.8
Df Residuals:                     123   BIC:                            -978.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0065      0.000    -13.796      0.0

In [24]:
ltn_precos['y'] = ltn_precos['r(1:t+2)'] - ltn_precos['r(1:t)']
ltn_precos['x'] = ltn_precos['f(3,2:t)'] - ltn_precos['r(1:t)']
result = sm.ols(formula="y ~ x", data=ltn_precos).fit()

Para $x=3$, a regressão é


$$
r(1:t+2) - r(1:t) = \alpha + \beta \left[f(3,2:t) - r(1:t)\right] + u_2(t+1)
$$

In [25]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 2.596e+04
Date:                Thu, 20 Oct 2022   Prob (F-statistic):          6.04e-145
Time:                        18:37:28   Log-Likelihood:                 410.27
No. Observations:                 125   AIC:                            -816.5
Df Residuals:                     123   BIC:                            -810.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0094      0.001     -9.747      0.0